In [1]:
import logging

from resimulate.euicc.card import Card
from resimulate.euicc.models.profile import Profile
from resimulate.euicc.mutation.engine import MutationEngine
from resimulate.euicc.recorder.recorder import OperationRecorder
from resimulate.euicc.transport.pcsc_link import PcscLink

logging.basicConfig(level=logging.DEBUG)

mutation_engine = MutationEngine()
recorder = OperationRecorder()

profile = Profile.from_activation_code("LPA:1$rsp.truphone.com$QR-G-5C-1LS-1W1Z9P7")

print(profile)

with PcscLink(recorder=recorder, apdu_data_size=255) as link:
    card = Card(link)
    print([app.name for app in card.supported_applications.values()])
    print(card.isd_r.get_euicc_challenge())
    print(card.isd_r.get_euicc_info_2())
    print(card.isd_r.get_euicc_info_1())
    print(card.isd_r.get_eid())
    print(card.isd_r.download_profile(profile))

recorder.save_file("test_get_euicc_info.resim")

DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Connecting to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00 using protocol T1
DEBUG:root:Connected to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Sent TPDU: 00a4040407a0000000871002, received data: , SW: 613f
DEBUG:root:Sent TPDU: 00c000003f, received data: 623d8202782183027fd0840ca0000000871002ff49ff0589a50c8001718304001754608701018a01058b032f0601c60f90017083010183018183010a83010b, SW: 9000


Profile(smdpp_address='rsp.truphone.com', matching_id='QR-G-5C-1LS-1W1Z9P7', confirmation_code=None)


DEBUG:root:Selected ADF a0000000871002
DEBUG:root:Found USIM via ADF a0000000871002 with CLA byte 00
DEBUG:root:Sent TPDU: 00a4040410a0000005591010ffffffff8900000100, received data: , SW: 6121
DEBUG:root:Sent TPDU: 00c0000021, received data: 6f1f8410a0000005591010ffffffff8900000100a5049f6501ffe0058203020300, SW: 9000
DEBUG:root:Selected ADF A0000005591010FFFFFFFF8900000100
DEBUG:root:Found ISDR via ADF A0000005591010FFFFFFFF8900000100 with CLA byte 00
DEBUG:root:Sent TPDU: 00a4040410a0000005591010ffffffff8900000200, received data: , SW: 611a
DEBUG:root:Sent TPDU: 00c000001a, received data: 6f188410a0000005591010ffffffff8900000200a5049f6501ff, SW: 9000
DEBUG:root:Selected ADF A0000005591010FFFFFFFF8900000200
DEBUG:root:Found ECASD via ADF A0000005591010FFFFFFFF8900000200 with CLA byte 00
DEBUG:root:Sent TPDU: 00a4040410a0000005591010ffffffff8900000300, received data: , SW: 6a82
DEBUG:root:Error selecting ADF A0000005591010FFFFFFFF8900000300! Failed with SW match failed! Expected 9000 an

['USIM', 'ISDR', 'ECASD']
f6fafef737c3374d5222bc0e07c5f817
{'profileVersion': b'\x02\x03\x01', 'svn': b'\x02\x03\x00', 'euiccFirmwareVer': b'$\x11\x04', 'extCardResource': b'\x81\x01\x0b\x82\x04\x00\x17R\x00\x83\x02\x7f\xe3', 'uiccCapability': (bytearray(b'\x7f6\xf7\xc0'), 26), 'ts102241Version': b'\x0f\x01\x00', 'globalplatformVersion': b'\x02\x03\x00', 'rspCapability': (bytearray(b'\x98'), 5), 'euiccCiPKIdListForVerification': [b'\x817\x0fQ%\xd0\xb1\xd4\x08\xd4\xc3\xb22\xe6\xd2^y[\xeb\xfb'], 'uiccCiPKIdListForSigning': [b'\x817\x0fQ%\xd0\xb1\xd4\x08\xd4\xc3\xb22\xe6\xd2^y[\xeb\xfb'], 'ppVersion': b'\x01\x00\x00', 'sasAcreditationNumber': 'KN-DN-UP-0924'}
{'svn': b'\x02\x03\x00', 'euiccCiPKIdListForVerification': [b'\x817\x0fQ%\xd0\xb1\xd4\x08\xd4\xc3\xb22\xe6\xd2^y[\xeb\xfb'], 'euiccCiPKIdListForSigning': [b'\x817\x0fQ%\xd0\xb1\xd4\x08\xd4\xc3\xb22\xe6\xd2^y[\xeb\xfb']}
b"\x89\x04@E!g'IH\x00\x00\x00\x015\x01\x87"


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): rsp.truphone.com:443
/home/niklas/Documents/documents/uni/master_thesis/resimulate/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rsp.truphone.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
DEBUG:urllib3.connectionpool:https://rsp.truphone.com:443 "POST /gsma/rsp2/es9plus/initiateAuthentication HTTP/1.1" 200 None
DEBUG:root:Initiate Authentication response: {"header":{"function_execution_status":{"status":"Executed-Success","status_code_data":null}},"transaction_id":"0000000018E39A33","server_signed_1":"MECACAAAAAAY45ozgRDC2nR33oZThy0CksUCus+wgxByc3AudHJ1cGhvbmUuY29thBBuz37ABPR/jTSr4m2vLENc","server_signature_1":"XzdAqHvdlY4V6nTWunwLwGXarOXvlTu9fzHuAghT3ugF3B3SODkXIgB1iHRS9oTSRrRWiAgZuLVMW+tCiB30flbeRQ==","euicc_ci_pki_to_be_u

{'seqNumber': 66, 'profileManagementOperation': (bytearray(b'\x80'), 1), 'notificationAddress': 'rsp.truphone.com', 'iccid': b'\x98DtV\x00\x00\x01\x98\x18\xf7'}


In [ ]:
from osmocom.utils import b2h
import base64

euicc_challenge = b"3]U\xbd\x90\xef\xbc\x9ddB\xfe\\>3\xa8X"
b2h(b"\x19\xf7^\xa5}\xe4\x01\x01\r\x0e<\xcbFtAS")

'19f75ea57de401010d0e3ccb46744153'

In [2]:
with PcscLink(recorder=recorder) as link:
    card = Card(link)
    recorder.replay("test_get_euicc_info.resim", card)

DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Connecting to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00 using protocol T1
DEBUG:root:Connected to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Error selecting ADF A0000005591010FFFFFFFF8900000300! Failed with SW match failed! Expected 9000 and got 6a82.
DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Replaying operation: get_euicc_challenge with mutation None
DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00


AttributeError: 'Operation' object has no attribute 'original_args'

In [2]:
import logging

from resimulate.scenarios import SCENARIOS
from resimulate.scenarios.models.scenario_runner import ScenarioRunner

logging.basicConfig(level=logging.DEBUG)

runner = ScenarioRunner(scenarios=SCENARIOS)
runner.run_scenarios()

DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Connecting to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00 using protocol T1
DEBUG:root:Connected to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Sent TPDU: 00a4040407a0000000871002, received data: , SW: 613f
DEBUG:root:Sent TPDU: 00c000003f, received data: 623d8202782183027fd0840ca0000000871002ff49ff0589a50c80017183040016d1208701018a01058b032f0601c60f90017083010183018183010a83010b, SW: 9000
DEBUG:root:Sent TPDU: 00a4040410A0000005591010FFFFFFFF8900000100, received data: , SW: 6121
DEBUG:root:Sent TPDU: 00c0000021, received data: 6f1f8410a0000005591010ffffffff8900000100a5049f6501ffe0058203020300, SW: 9000
DEBUG:root:Sent TPDU: 00a4040410A0000005591010FFFFFFFF8900000200, received data: , SW: 611a
DEBUG:root:Sent TPDU: 00c000001a, received data: 6f188410a000000559101

In [2]:
import asn1tools

rsp_v2_6 = asn1tools.compile_files(
    [
        "/home/niklas/Documents/documents/uni/master_thesis/resimulate/asn/pkix1_explicit_88.asn",
        "/home/niklas/Documents/documents/uni/master_thesis/resimulate/asn/pkix1_implicit_88.asn",
        "/home/niklas/Documents/documents/uni/master_thesis/resimulate/asn/pe_definitions_v3_4.asn",
        "/home/niklas/Documents/documents/uni/master_thesis/resimulate/asn/rsp_definitions_v2_6.asn",
    ],
    codec="ber",
    cache_dir="asn_cache",
)

In [5]:
import base64

authenticate_server_response = "vziCBeigggXkMIHJgAgAAAAAGNjLsoMQcnNwLnRydXBob25lLmNvbYQQa/Ugq/w3o10ewVKyNwVpQ78id4EDAgMBggMCAwCDAyQRBIQNgQELggQAF1IAgwJ/44UFBn8298CGAw8BAIcDAgMAiAIDmKkWBBSBNw9RJdCx1AjUw7Iy5tJeeVvr+6oWBBSBNw9RJdCx1AjUw7Iy5tJeeVvr+wQDAQAADA1LTi1ETi1VUC0wOTI0oB+AE1FSLUctNUMtMUxTLTFXMVo5UDehCIAENSkGEaEAXzdAUzHVWCNLuyt5pDB6GQb4jFdcoj5/oAxegG7jj46J8XDYgJzxms+uLc70lkIlrfCLc9HAgpKWlXGqocsWTh3wajCCAgcwggGsoAMCAQICDgRj5NPd16wGH8wb7ZorMAoGCCqGSM49BAMCMGIxCzAJBgNVBAYTAlVLMRIwEAYDVQQHDAlDYW1icmlkZ2UxDjAMBgNVBAoMBUtpZ2VuMRQwEgYDVQQLDAtFbmdpbmVlcmluZzEZMBcGA1UEAwwQRVVNQ29uc3VtZXJER0R1YjAgFw0yNDExMTIwODMxMTdaGA8yMTI0MTExMjA4MzExN1owOzEOMAwGA1UECgwFS2lnZW4xKTAnBgNVBAUTIDg5MDQ0MDQ1MjE2NzI3NDk0ODAwMDAwMDAxMzUwMTg3MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEqt+MxSB28dwnDlymxPJurDV+ulj7VdM2cQQHGQDg8u33HNd//+POlr7FnJu3D7+P5FutkbX6da0HATAKAIevHKNrMGkwHQYDVR0OBBYEFJ8DsA/PJVMFhZJtcw1QAzYWtBOaMB8GA1UdIwQYMBaAFBiaCFOv1JT2sHmaC2fw9dDBKeP2MA4GA1UdDwEB/wQEAwIHgDAXBgNVHSABAf8EDTALMAkGB2eBEgECAQEwCgYIKoZIzj0EAwIDSQAwRgIhAOQGFqozk99v9eE13TWQWtTN4Ytwstk2mYZLBIwzujX7AiEAgR96Fg7h9DDqpzBiiLChSF8eudV6SpLzABtCi6NZL3gwggLGMIICbaADAgECAhBazo+Y3IZJQEq9rWV+IFCTMAoGCCqGSM49BAMCMEQxGDAWBgNVBAoTD0dTTSBBc3NvY2lhdGlvbjEoMCYGA1UEAxMfR1NNIEFzc29jaWF0aW9uIC0gUlNQMiBSb290IENJMTAeFw0yMjEwMDUwMDAwMDBaFw00OTEyMzEyMzU5NTlaMGIxCzAJBgNVBAYTAlVLMRIwEAYDVQQHDAlDYW1icmlkZ2UxDjAMBgNVBAoMBUtpZ2VuMRQwEgYDVQQLDAtFbmdpbmVlcmluZzEZMBcGA1UEAwwQRVVNQ29uc3VtZXJER0R1YjBZMBMGByqGSM49AgEGCCqGSM49AwEHA0IABMb5fuovdorRe6himrAE7PeQ/0NX8ggb4VwjGfcuyWb5omC7tcc1FgaXd3uCm1nJ9l+8mj14VBDYmG/CTvikO1ujggEhMIIBHTASBgNVHRMBAf8ECDAGAQH/AgEAMBcGA1UdIAEB/wQNMAswCQYHZ4ESAQIBAjBNBgNVHR8ERjBEMEKgQKA+hjxodHRwOi8vZ3NtYS1jcmwuc3ltYXV0aC5jb20vb2ZmbGluZWNhL2dzbWEtcnNwMi1yb290LWNpMS5jcmwwDgYDVR0PAQH/BAQDAgEGMDcGA1UdHgEB/wQtMCugKTAnpCUwIzEOMAwGA1UEChMFS2lnZW4xETAPBgNVBAUTCDg5MDQ0MDQ1MBYGA1UdEQQPMA2ICysGAQQBg7sTAQICMB0GA1UdDgQWBBQYmghTr9SU9rB5mgtn8PXQwSnj9jAfBgNVHSMEGDAWgBSBNw9RJdCx1AjUw7Iy5tJeeVvr+zAKBggqhkjOPQQDAgNHADBEAiAQZhiqj72l9CmSgfTjcSIE5unzsJXxm45S7aYffkpNsQIgNfgtDjRuLoNsfxhEOSo5LkPb3vpE4L8iFSXolOWTg9E="
data = base64.b64decode(authenticate_server_response)
response = db.types["AuthenticateServerResponse"]
db.encode("GetEuiccChallengeRequest", dict())

b'\xbf.\x00'

In [2]:
import base64
import binascii

b64_certificate = "MIICVDCCAfugAwIBAgIQZumBlNebG7wHl/5VrnPCxDAKBggqhkjOPQQDAjBEMRgwFgYDVQQKEw9HU00gQXNzb2NpYXRpb24xKDAmBgNVBAMTH0dTTSBBc3NvY2lhdGlvbiAtIFJTUDIgUm9vdCBDSTEwHhcNMjQwOTE4MDAwMDAwWhcNMjcwOTE3MjM1OTU5WjBCMSAwHgYDVQQKDBdSRURURUEgTU9CSUxFIFBURS4gTFRELjEeMBwGA1UEAwwVUkVEVEVBIE1PQklMRSBEUF9BVVRIMFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEJ1JBe+EEsyocAQ9vYzjCBttMkGcufGsLiULDkIHcbGKiSBHRkZIPCI8S/Rhw6yB1BDW0IUA0FY4eRVdsqS2tNqOB0DCBzTAXBgNVHSABAf8EDTALMAkGB2eBEgECAQQwTQYDVR0fBEYwRDBCoECgPoY8aHR0cDovL2dzbWEtY3JsLnN5bWF1dGguY29tL29mZmxpbmVjYS9nc21hLXJzcDItcm9vdC1jaTEuY3JsMA4GA1UdDwEB/wQEAwIHgDATBgNVHREEDDAKiAgrBgEEAYPFHDAdBgNVHQ4EFgQUH3INoRziBpuXcwV1oNpnXGeVt6IwHwYDVR0jBBgwFoAUgTcPUSXQsdQI1MOyMubSXnlb6/swCgYIKoZIzj0EAwIDRwAwRAIgbq6dUp9M0LSTAVTgmiPixvn/lvcIoMnc7iJ13DLmZ4oCIHIdk6ZjtCVfahek5h8yQ0WdZ646hPRd5M3SpReTZNWJ"
certificate = base64.b64decode(b64_certificate)
decoded = rsp_v2_6.decode("Certificate", certificate)
encoded = rsp_v2_6.encode("Certificate", decoded)

hex_encoded = binascii.hexlify(encoded)
hex_certificate = binascii.hexlify(certificate)
assert hex_encoded == hex_certificate, (
    f"Decoded and encoded certificate do not match: {hex_certificate} != {hex_encoded}"
)
print(decoded)


AssertionError: Decoded and encoded certificate do not match: b'30820254308201fba003020102021066e98194d79b1bbc0797fe55ae73c2c4300a06082a8648ce3d040302304431183016060355040a130f47534d204173736f63696174696f6e312830260603550403131f47534d204173736f63696174696f6e202d205253503220526f6f7420434931301e170d3234303931383030303030305a170d3237303931373233353935395a30423120301e060355040a0c17524544544541204d4f42494c45205054452e204c54442e311e301c06035504030c15524544544541204d4f42494c452044505f415554483059301306072a8648ce3d020106082a8648ce3d030107034200042752417be104b32a1c010f6f6338c206db4c90672e7c6b0b8942c39081dc6c62a24811d191920f088f12fd1870eb20750435b4214034158e1e45576ca92dad36a381d03081cd30170603551d200101ff040d300b3009060767811201020104304d0603551d1f044630443042a040a03e863c687474703a2f2f67736d612d63726c2e73796d617574682e636f6d2f6f66666c696e6563612f67736d612d727370322d726f6f742d6369312e63726c300e0603551d0f0101ff04040302078030130603551d11040c300a88082b0601040183c51c301d0603551d0e041604141f720da11ce2